<a href="https://colab.research.google.com/github/MatveiV/ML_Fin_Notebooks/blob/main/%D0%A2%D0%B5%D0%BC%D0%B0_14_1_BigData_%D0%B4%D0%BB%D1%8F_%D1%84%D0%B8%D0%BD%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%BE%D0%B3%D0%BE_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install asyncmy -qq

In [ ]:
import asyncio
import os
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%d.%m.%Y %H:%M:%S')

from asyncmy import connect
from asyncmy.cursors import DictCursor
import asyncmy
import asyncio
import uuid

async def insert_rows(conn, cursor):
    while True:
        await cursor.execute(f"INSERT INTO default_db.financial_data(`id`,`create_time`) VALUES ('{uuid.uuid4()}','');"*1000)
        await conn.commit()

async def get_stats(conn, cursor):
    while True:
        await cursor.execute("SELECT COUNT(`id`) AS id FROM default_db.financial_data;")
        result = await cursor.fetchone()
        logging.info(f"Count: {result['id']}")

        await cursor.execute("SELECT MAX(`id`) AS id FROM default_db.financial_data;")
        result = await cursor.fetchone()
        logging.info(f"Max id: {result['id']}")

async def run():
  try:
    conn = await connect(user = "gen_user",
            password = "L.5O1etKjGcw>u",
            host = "94.241.139.196",
            database = "default_db",
            port = 3306)
    if conn:
      logging.info("Успешно создано соединение с базой данных")

    async with conn.cursor(cursor=DictCursor) as cursor:
        await cursor.execute("""
                            CREATE TABLE IF NOT EXISTS `default_db`.`financial_data` (
                            `id` VARCHAR(40) NOT NULL,
                            `create_time` TIMESTAMP NULL DEFAULT CURRENT_TIMESTAMP
                            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci;
                            """.strip())
        logging.info("Успешно создана таблица")
        tasks = [insert_rows(conn, cursor), get_stats(conn, cursor)] #бесконечный цикл записи в таблицу
        await asyncio.gather(*tasks)

  except Exception as e:
    logging.error(e, exc_info=True)


if __name__ == "__main__":
  try:
      loop = asyncio.get_running_loop()
  except RuntimeError:
      loop = None
  if loop and loop.is_running():
      print('Добавляем корутину в event loop.')
      tsk = loop.create_task(run())
      tsk.add_done_callback(lambda t: logging.info(f'Задача выполнена={t.result()}'))
  else:
      logging.info('Запускаем новый event loop')
      result = asyncio.run(run())

Добавляем корутину в event loop.
